# Minicurso: O que o twitter está pensando?

Estes tutoriais apresentam os principais scritps desenvolvidos no minicurso: **O que o twitter está pensando? Extraindo informações em redes sociais utilizando Python**. Os arquivos completos dos scritps e códigos gerados podem ser encontrados nas pastas **scritps** e **web** na raiz do repositório.

A apresentação referente a este minicurso está disponível no site: http://www.data2learning.com/cursos.

## 04 - Juntando tudo

Ao longo deste tutorial aprendemos como selecionar dados do twitter e fazer algumas as etapas de pré-processamento. O objetivo desta seção é juntar tudo. Vamos coletar dados do twitter, pré-processar e imprimir a lista de tokens mais frequentes para um conjunto de dados coletados.

* O primeiro passo é coletar tweets e adicionar em uma lista. Para nossa tarefa só interessa o texto e o login do usuário que está postando. Demais informações podem ser descartadas.
* Em seguida vamos processar os textos coletados tokenizando.
* Deve ser armazenado uma lista das hashtags mais postadas e dos usuários que mais postaram.
* Em seguida devemos eliminar as stopwords.
* Devemos eliminar eventuais problemas no texto: palavras repetidas e escritas de forma incorreta.
* Por fim, extrair os 50 termos mais frequentes usando bigram e trigram.
* Ao final deve imprimir a lista dos termos, a lista das hashtags e a lista dos usuários.

In [1]:
# Importando as bibliotecas necessárias

from twython import Twython
from datetime import datetime

# Definindo as chaves do Twitter

APP_KEY = "0rBTp9a35qIvA5ufGpxPGNkWu"
APP_SECRET = "rGStqnwrDjuzo1zwnXjpPlrilvDmvNljhRh6cTs1pG48K6ZLG6"
OAUTH_TOKEN = "736392442384154624-blYbsB4awwSezrNUH7L5jTG6JPglJy3"
OAUTH_TOKEN_SECRET = "bvPr7Y8BQeyN46UKZtDJPyP0Bx4Y8IuDRDVYxsc3LNAlb"

tw = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

result = tw.search(q="futebol",count=100,lang="pt")

tweets = result['statuses']

list_text = []
list_user = []

for tweet in tweets:
    
    #Evita pegar texto truncado no caso de Retweeteds
    if 'retweeted_status' in tweet.keys():
        list_text.append(tweet['retweeted_status']['text'])
    else:
        list_text.append(tweet['text'])
        
    list_user.append(tweet['user']['screen_name'])    

As variáveis **list_text** e **list_user** armazenam a lista de todos os textos e usuários, respectivamente. O passo seguinte é tratar essas listas com as etapas de pré-processamento.

Vamos tratar todos os tweets como um único texto e fazer o pré-processamento nesse texto. A partir desse texto vamos extrair a lista palavras mais citadas, lista de usuários citados e lista de hashtags. 

In [3]:
from unicodedata import normalize, category
from nltk.tokenize import regexp_tokenize
from collections import Counter, Set
from nltk.corpus import stopwords


pattern = r'(https://[^"\' ]+|www.[^"\' ]+|http://[^"\' ]+|\w+|\@\w+|\#\w+)'
portuguese_stops = stopwords.words(['portuguese'])

users_cited = []
links_appears = []
hashtags = []

patterns = []

for text in list_text:
    
    text = text.lower()
    
    patterns += regexp_tokenize(text, pattern)

    users_cited += [e for e in patterns if e[0] == '@']
    links_appears += [e for e in patterns if e[:4] == 'http']
    hashtags += [e for e in patterns if e[0] == '#']
    
    final_tokens = [e for e in patterns if e[:4] != 'http']
    final_tokens = [e for e in final_tokens if e[:4] != 'www.']
    final_tokens = [e for e in final_tokens if e[0] != '#']
    final_tokens = [e for e in final_tokens if e[0] != '@']
    
    
words = [word for word in final_tokens if word not in portuguese_stops]

word_set = set(words)

In [4]:
#Classes para correção
import re
import enchant
from nltk.metrics import edit_distance

class RepeatReplacer(object):
    
    def __init__(self):
        self.repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
        self.repl = r'\1\2\3'
        
    def replace(self, word):
        repl_word = self.repeat_regexp.sub(self.repl, word)

        if repl_word != word:
            return self.replace(repl_word)
        else:
            return repl_word

class SpellingReplacer(object):
    
    def __init__(self, dict_name='pt_BR',max_dist=2):
        self.spell_dict = enchant.Dict(dict_name)
        self.max_dist = max_dist
        
    def replace(self, word):
        
        if self.spell_dict.check(word):
            return word
        
        suggestions = self.spell_dict.suggest(word)
        
        if suggestions and edit_distance(word, suggestions[0]) <= self.max_dist:
            return suggestions[0]
        else:

            return word

In [6]:
#Cria um map das palavras erradas para as palavras corrigidas

replacer_spelling = SpellingReplacer()
replacer_repeat = RepeatReplacer()

new_words = []
map_words = {}
for word in word_set:
    
    new_word = replacer_repeat.replace(word)
    new_word = replacer_spelling.replace(new_word)
    
    map_words[word] = new_word

In [8]:
words_temp = [map_words[word] for word in words ]
words_temp = [word for word in words_temp if len(word) >= 3]

final_words = []

for word in words_temp:
    try:
        new_word = normalize('NFKD', word.lower()).encode('ASCII','ignore')
    except UnicodeEncodeError:
        new_word = normalize('NFKD', word.lower().decode('utf-8')).encode('ASCII','ignore')

    final_words.append(new_word)

In [9]:
import nltk

frequence_terms = nltk.FreqDist(final_words)

for word in frequence_terms.most_common(10):
    print word[0]

futebol
transito
comemorar
rio
galera
pleno
abriu
leilton
sensacional
solar


In [10]:
# Retornando os usuários, usuários citados e hashtags mais frequentes
frequence_users = nltk.FreqDist(list_user)
frequence_users_cited = nltk.FreqDist(users_cited)
frequence_hashtags = nltk.FreqDist(hashtags)

print(frequence_users.most_common(10))
print(frequence_users_cited.most_common(10))
print(frequence_hashtags.most_common(10))

[(u'MiguelCampos12', 4), (u'dudalbuquerqu40', 2), (u'Wallacefla1981', 2), (u'capteinswag', 2), (u'Delatorres10', 2), (u'tmlinsonxx', 1), (u'silva_viviann', 1), (u'Brenda__Evelin', 1), (u'insanomeo', 1), (u'jooaovpereiraa', 1)]
[(u'@cbf_futebol', 517), (u'@uolesporte', 212), (u'@uol', 212), (u'@dan_nepomuceno', 184), (u'@italoaraujo08', 146), (u'@youtube', 98), (u'@catleticomg', 95), (u'@whyalef', 81), (u'@rockfla81', 32), (u'@deldrd', 32)]
[(u'#onedirection', 135), (u'#mpn', 135), (u'#copapasion', 100), (u'#ariasjustinbieber', 80), (u'#globoesporte', 61), (u'#louisfav', 21), (u'#lideran\xe7aemjogo', 8)]


In [11]:
#Pegando os bigram e trigram mais frequentes

from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder
from nltk.metrics import BigramAssocMeasures, TrigramAssocMeasures

bcf = BigramCollocationFinder.from_words(final_words)
tcf = TrigramCollocationFinder.from_words(final_words)

bcf.apply_freq_filter(3)
tcf.apply_freq_filter(3)

#result_bi = bcf.nbest(BigramAssocMeasures.likelihood_ratio, 5)
#result_tri = tcf.nbest(TrigramAssocMeasures.likelihood_ratio, 5)

result_bi = bcf.nbest(BigramAssocMeasures.raw_freq, 5)
result_tri = tcf.nbest(TrigramAssocMeasures.raw_freq, 5)


for r in result_bi:
    print(r)
print
for r in result_tri:
    print(r)


('abriu', 'teto')
('comemorar', 'galera')
('futebol', 'sensacional')
('galera', 'pleno')
('leilton', 'abriu')

('abriu', 'teto', 'solar')
('comemorar', 'galera', 'pleno')
('galera', 'pleno', 'transito')
('leilton', 'abriu', 'teto')
('pleno', 'transito', 'futebol')


Com isso finalizamos nossa parte de pré-processamento de texto. Observe que só em extrair de forma correta os termos mais frequentes temos uma visão melhor do que está sendo discutido nas redes sociais. No entanto, exibir isso somente na linha de comando não é muito atrativo. O próximo passo é exibir estas informações em uma página web.